In [ ]:
import os
import os.path as osp
import argparse
from cv2 import imread, normalize, resize
import cv2
import json
import numpy as np
import time
import datetime
import random
import logging
import sys
from PIL import Image

from utils import euclidean_metric, one_hot, count_acc
from utils import pprint, set_gpu, ensure_path, AverageMeter, Timer, accuracy, one_hot

import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch import nn, optim, autograd
from torch.utils.data import DataLoader
import torch.utils.data as data
import torchvision
from random import sample, random

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset, Dataset, TensorDataset, WeightedRandomSampler
import os.path as osp
from tqdm import tqdm

from random import sample, random
import imgaug.augmenters as iaa

batch_size = 32
num_epoch = 50

In [ ]:
class Customdataset(data.Dataset):
    def __init__(self,transform = None):
        dalist=None
        self.file_paths = []
        self.label = []
        self.transform = transform

        file_path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/train.1-1.txt' # 폴더 경로

        with open(file_path) as f:
            lines = f.readlines()

        image_names = [line.rstrip('\n') for line in lines]
        
                
        with open('/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/annotation/single_image.6class.json', 'r') as json_file:
            data = json.load(json_file)
            
        path_image = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/image/"
        files = os.listdir(path_image)
        
        for file in files:
            if file in image_names:
                self.label.append(data['single_image'][file]['class'][0])
                self.file_paths.append(path_image + '/' + file)

    def __len__(self):

        return len(self.file_paths)


    def __getitem__(self, idx):
        
        label = self.label[idx]
        idx_num = idx
        tmp_prob = np.random.rand()
        weight_dict = {"0" :  (400/250) / (400/250 + 400 * 5 / 30), "1" : (400/30) / (400/250 + 400 * 5 / 30), "2" : (400/30) / (400/250 + 400 * 5 / 30), 
                       "3" : (400/30) / (400/250 + 400 * 5 / 30), "4" : (400/30) / (400/250 + 400 * 5 / 30), "5" : (400/30) / (400/250 + 400 * 5 / 30)}        
        
        if weight_dict[str(label)] < tmp_prob:
            while True : 
                idx_num = np.random.randint(len(self.file_paths))
                tmp_prob =  np.random.rand()
                label = self.label[idx_num]
                
                if weight_dict[str(label)] > tmp_prob:
                    break

        path = self.file_paths[idx_num]
        image = Image.open(path).convert('RGB')
        
          
        if self.transform:
            image = self.transform(image)
        
        return image, int(label)

In [ ]:
class ImgAugTransform:
  def __init__(self):
    self.aug = iaa.Sequential([
                iaa.BlendAlpha((0, 0.25),
                iaa.Affine(rotate=(-25, 25))),
                iaa.GaussianBlur(sigma=(0, 2)),
                ])
      
  def __call__(self, img):
    img = np.array(img).T
    img = self.custom_lightness(img, 8 * np.random.randn())
    img = self.aug.augment_image(img)
    return img.T

  def custom_lightness(self, image, custom_sigma):
    img = image
    img_YUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)    
    y = img_YUV[:,:,0]    
    
    rows = y.shape[0]    
    cols = y.shape[1]
    
    ### illumination elements와 reflectance elements를 분리하기 위해 log를 취함
    imgLog = np.log1p(np.array(y, dtype='float') / 255) # y값을 0~1사이로 조정한 뒤 log(x+1)
    
    ### frequency를 이미지로 나타내면 4분면에 대칭적으로 나타나므로 
    ### 4분면 중 하나에 이미지를 대응시키기 위해 row와 column을 2배씩 늘려줌
    M = 2*rows + 1
    N = 2*cols + 1
    
    ### gaussian mask 생성 sigma = 10
    sigma = custom_sigma
    (X, Y) = np.meshgrid(np.linspace(0, N-1, N), np.linspace(0, M-1, M)) # 0~N-1(and M-1) 까지 1단위로 space를 만듬
    Xc = np.ceil(N/2) # 올림 연산
    Yc = np.ceil(M/2)
    gaussianNumerator = (X - Xc)**2 + (Y - Yc)**2 # 가우시안 분자 생성
    
    ### low pass filter와 high pass filter 생성
    LPF = np.exp(-gaussianNumerator / (2*sigma*sigma))
    HPF = 1 - LPF
    
    ### LPF랑 HPF를 0이 가운데로 오도록iFFT함. 
    ### 사실 이 부분이 잘 이해가 안 가는데 plt로 이미지를 띄워보니 shuffling을 수행한 효과가 났음
    ### 에너지를 각 귀퉁이로 모아 줌
    LPF_shift = np.fft.ifftshift(LPF.copy())
    HPF_shift = np.fft.ifftshift(HPF.copy())
    
    ### Log를 씌운 이미지를 FFT해서 LPF와 HPF를 곱해 LF성분과 HF성분을 나눔
    img_FFT = np.fft.fft2(imgLog.copy(), (M, N))
    img_LF = np.real(np.fft.ifft2(img_FFT.copy() * LPF_shift, (M, N))) # low frequency 성분
    img_HF = np.real(np.fft.ifft2(img_FFT.copy() * HPF_shift, (M, N))) # high frequency 성분
    
    ### 각 LF, HF 성분에 scaling factor를 곱해주어 조명값과 반사값을 조절함
    gamma1 = 0.3
    gamma2 = 1.5
    img_adjusting = gamma1*img_LF[0:rows, 0:cols] + gamma2*img_HF[0:rows, 0:cols]
    
    ### 조정된 데이터를 이제 exp 연산을 통해 이미지로 만들어줌
    img_exp = np.expm1(img_adjusting) # exp(x) + 1
    img_exp = (img_exp - np.min(img_exp)) / (np.max(img_exp) - np.min(img_exp)) # 0~1사이로 정규화
    img_out = np.array(255*img_exp, dtype = 'uint8') # 255를 곱해서 intensity값을 만들어줌
    
    ### 마지막으로 YUV에서 Y space를 filtering된 이미지로 교체해주고 RGB space로 converting
    img_YUV[:,:,0] = img_out
    result = cv2.cvtColor(img_YUV, cv2.COLOR_YUV2BGR)
    
    return result

aug_transforms = ImgAugTransform()

data_transforms = transforms.Compose([
              
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        aug_transforms
        
        ])



In [ ]:
##커스텀 데이터셋
dataset = Customdataset(transform=data_transforms)
train_size = int(0.8 * len(dataset))
print(f"train size :{train_size}")
val_size = len(dataset) - train_size
print(f"test_size :{val_size}")
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(f"train_dataset :{len(train_dataset)}")
print(f'test dataset :{len(val_dataset)}')


In [ ]:
from torchsampler import ImbalancedDatasetSampler

train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size = batch_size, #배치사이즈
                                               shuffle = True
                                               )

val_loader = torch.utils.data.DataLoader(val_dataset,
                                               batch_size = batch_size,                                              
                                               shuffle = True
                                              )

In [ ]:
from torchvision import models
import torch

resnet50_pretrained = models.resnet50(pretrained=True)

num_classes = 6
num_ftrs = resnet50_pretrained.fc.in_features
resnet50_pretrained.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device('cuda:0')
resnet50_pretrained.to(device)
print()

In [ ]:
from pytorchtools import EarlyStopping # 위 링크의 깃허브 파일에서 임포트

early_stopping = EarlyStopping(patience = 3, verbose = True, min_epoch=30)

loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(resnet50_pretrained.parameters(), lr=0.00005)

train_loss_arr =[]
val_loss_arr =[]
n = len(train_loader)

model_path = '/home/iai/Desktop/SH/cognex/model/'

for i in range(num_epoch):
    
    train_running_loss = 0.0
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y =  label.to(device)
        
        optimizer.zero_grad()
        
        output = resnet50_pretrained.forward(x)
        
        loss = loss_func(output,y)
        loss.backward()
        optimizer.step()
        train_running_loss += loss.item()
    
    train_loss_arr.append(train_running_loss / n)
        
    resnet50_pretrained.eval()
    val_running_loss = 0.0
    for j,[image,label] in enumerate(val_loader):
        
        x = image.to(device)
        y =  label.to(device)
        
        y_pred = resnet50_pretrained(x)
        loss = loss_func(y_pred, y)

        val_running_loss += loss.item()
    
    val_loss_arr.append(val_running_loss / len(val_loader))       
    early_stopping(val_running_loss, resnet50_pretrained)
    torch.save(resnet50_pretrained, model_path + str(i) + 'epoch_model.pt')
    
    print('[%d] train_loss: %.5f , val_loss: %.5f ' %(i + 1, train_running_loss / len(train_loader) , val_running_loss / len(val_loader)))
        
    if early_stopping.early_stop and i > 20: # 조건 만족 시 조기 종료
        break


In [ ]:
import matplotlib.pyplot as plt


plt.plot(train_loss_arr,
         color='skyblue',
         marker='o', markerfacecolor='blue',
         markersize=4)

plt.plot(val_loss_arr,
         color='pink',
         marker='x', markerfacecolor='red',
         markersize=4)

plt.title('Model loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
class Customdataset(torch.utils.data.Dataset):
    def __init__(self,transform = None):
        dalist=None
        self.file_paths = []
        self.label = []
        self.context = []
        self.transform = transform

        file_path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/test.1.txt' # 폴더 경로

        with open(file_path) as f:
            lines = f.readlines()

        image_names = [line.rstrip('\n') for line in lines]


        path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/image' # 폴더 경로
        files = os.listdir(path) # 해당 폴더에 있는 파일 이름을 리스트 형태로 받음
        
        with open('/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/annotation/single_image.6class.json', 'r') as json_file:
            data = json.load(json_file)  
    
        for file in files:
            if file in image_names:
                self.label.append(data['single_image'][file]['class'][0])
                self.file_paths.append(path + '/' + file)

    def __len__(self):

        return len(self.file_paths)


    def __getitem__(self, idx):
        path = self.file_paths[idx]
        image = Image.open(path).convert('RGB')
        label = self.label[idx]

        if self.transform is not None:
            image = self.transform(image)
        
        return image, int(label)
    

data_transforms = transforms.Compose([
        
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),

        ])

    
test_dataset = Customdataset(transform=data_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size = 32                                       
                                              )

In [ ]:
resnet50_pretrained = models.resnet50(pretrained=True)

num_classes = 6
num_ftrs = resnet50_pretrained.fc.in_features
resnet50_pretrained.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device('cuda:0')
resnet50_pretrained.to(device)

best_model_epoch = val_loss_arr.index(min(val_loss_arr))
resnet50_pretrained = torch.load(model_path + str(best_model_epoch) +'epoch_model.pt')

In [ ]:
correct = 0
total = 0
pred_ls = []
real_ls = []

context_ls = [] 

with torch.no_grad(): # 파라미터 업데이트 같은거 안하기 때문에 no_grad를 사용.
  # net.eval() # batch normalization이나 dropout을 사용하지 않았기 때문에 사용하지 않음. 항상 주의해야함.
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device) 
    outputs = resnet50_pretrained(images)
    _, predicted = torch.max(outputs.data, 1) # 6개의 class중 가장 값이 높은 것을 예측 label로 추출.
    total += labels.size(0) # test 개수
    correct += (predicted == labels).sum().item() # 예측값과 실제값이 맞으면 1 아니면 0으로 합산.
    
    pred_ls.append(predicted.cpu().numpy())
    real_ls.append(labels.cpu().numpy())
    
    #context_ls.append(context)

print(f'accuracy of 1600 test images: {100*correct/total}%')

In [ ]:
pred = np.array(pred_ls).reshape(-1)
real = np.array(real_ls).reshape(-1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(pred, real))

In [ ]:
import itertools

def plot_confusion_matrix(con_mat, labels, title='Confusion Matrix', cmap=plt.cm.get_cmap('Blues'), normalize=False):
    plt.imshow(con_mat, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks = np.arange(len(labels))
    nlabels = []
    for k in range(len(con_mat)):
        n = sum(con_mat[k])
        nlabel = '{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)
    plt.xticks(marks, labels)
    plt.yticks(marks, nlabels)

    thresh = con_mat.max() / 2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
cm_df = confusion_matrix(pred, real)    
label=['ok', 'scratch', 'fm', 'pin', 'dent', 'glue']
plot_confusion_matrix(cm_df, labels=label, normalize=False)